In [1]:
# Select the review that bests represents a given topic
# Select the business with the highest proportion of a given topic

import cPickle as pickle
from gensim import corpora, models, similarities, utils
import numpy as np
import pandas as pd
from KAsql import query_SQL
import pyLDAvis.gensim
import pyLDAvis

In [2]:
lda = pickle.load(open('lda_ortho_30t_pos.p','rb'))
lda.print_topics()

[u'0.010*knee + 0.010*staff + 0.008*office + 0.008*recommend + 0.008*excellent + 0.007*great + 0.007*never + 0.006*time + 0.006*questions + 0.006*better',
 u'0.008*ive + 0.007*hip + 0.007*never + 0.007*time + 0.006*staff + 0.006*even + 0.006*great + 0.006*first + 0.005*care + 0.005*hes',
 u'0.010*years + 0.008*office + 0.008*time + 0.007*knee + 0.005*staff + 0.005*really + 0.005*work + 0.005*appointment + 0.005*questions + 0.005*best',
 u'0.010*wrist + 0.008*great + 0.008*staff + 0.007*time + 0.006*didnt + 0.006*arm + 0.006*really + 0.005*ive + 0.005*appointment + 0.005*minutes',
 u'0.013*knee + 0.010*time + 0.009*staff + 0.008*office + 0.008*highly + 0.008*recommend + 0.007*experience + 0.005*first + 0.005*friendly + 0.005*care',
 u'0.009*office + 0.007*time + 0.007*great + 0.006*another + 0.006*injury + 0.005*mri + 0.005*months + 0.005*weeks + 0.005*going + 0.004*told',
 u'0.011*time + 0.008*hes + 0.007*good + 0.007*treatment + 0.006*staff + 0.006*best + 0.005*dont + 0.005*need + 0.0

In [15]:
# Load dfs with gammas for all reviews and averaged by business/doctor
# All reviews

bid_df = pd.read_pickle('bid_tmeans.p')#index of this df is the bid, but can't be indexed by 'BID'
Reviews_topics_df = pd.read_pickle('Review_topics_df.p')

topic_word_df = pd.read_pickle('topic_word_df.p')
names_df = pd.read_pickle('names_df.p')

In [37]:
print Reviews_topics_df.head()#RID and BID are first two columns then topic weights

#print bid_df.ix[309,:]
#print bid_df#bid is the index and the topic numbers are columns - topic weights are values


     RID  BID  0  1  2  3  4  5  6  7 ...  20  21        22  23        24  25  \
0  20406  303  0  0  0  0  0  0  0  0 ...   0   0  0.000000   0  0.000000   0   
1  20407  303  0  0  0  0  0  0  0  0 ...   0   0  0.000000   0  0.539521   0   
2  20408  303  0  0  0  0  0  0  0  0 ...   0   0  0.000000   0  0.992267   0   
3  20409  303  0  0  0  0  0  0  0  0 ...   0   0  0.989139   0  0.000000   0   
4  20410  303  0  0  0  0  0  0  0  0 ...   0   0  0.000000   0  0.000000   0   

   26  27  28  29  
0   0   0   0   0  
1   0   0   0   0  
2   0   0   0   0  
3   0   0   0   0  
4   0   0   0   0  

[5 rows x 32 columns]


In [95]:
def three_representative_reviews(mytopic,Reviews_topics_df):
    """What 3 reviews best represent a given topic?
    Those with the highest topic-weights on one topic
    Revise to print top 3 reviews and see if they are consistent and validate with stars
    - use these reviews to help define & validate each topic"""

    #Check what df looks like prior to sorting

    # Sort on a given topic
    mysorted = Reviews_topics_df.sort(mytopic, axis=0, ascending=False)
    idx = mysorted.index[0:3]# get index of top-ranking review for mytopic

    RID = mysorted.loc[idx]['RID']#Note that if you replace idx with 0, and print the results, it is not the same
    RIDs = [str(int(value)) for value in RID.values]#list of strings (use int to get rid of decimal)

    #For debugging
    print 'Topic: ',mytopic,'\n'
    print 'Index: ',idx
    print 'RID: ',RID.values,'\n'#type Series
    
    # Pull 3 reviews, and see if they appear to represent one topic
    #from KAsql import query_SQL

    # SQL query uses my function
    #from KAsql import query_SQL
    sql = 'SELECT id, stars, comment FROM review WHERE id='+RIDs[0]+' or id='+RIDs[1]+' or id='+RIDs[2]+';'
    rows = query_SQL(sql)

    sqlRID=[];stars=[];comment=''
    sqlRID = rows[:][0]
    stars = rows[:][1]
    comment = rows[:][2]

    # Print Statements
    print sql,'\n'
    print '\nTop three reviews most related to topic: ',mytopic,'\n'
    print mysorted.head(3),'\n'#Confirm RIDs match with sql data

    # For some reason rows are printed in opposite order from listing
    texts=[]
    for i,row in enumerate(rows):
        texts.append([row[0],row[1],row[2]])
    
    #print row[2][:500]
    return texts
    

In [102]:
# Kenneth Akizuki, bid 309
# High on topics 17 and 11 and 22, although well-rounded
texts= three_representative_reviews(19,Reviews_topics_df)

write1 = str(texts[0])
write2 = str(texts[2])


print texts


Topic:  19 

Index:  Int64Index([667, 1178, 59], dtype='int64')
RID:  [21073 21584 20465] 

USE ortho;
SELECT id, stars, comment FROM review WHERE id=21073 or id=21584 or id=20465; 


Top three reviews most related to topic:  19 

        RID  BID  0  1  2  3  4  5  6  7 ...  20  21  22  23  24  25  26  27  \
667   21073  330  0  0  0  0  0  0  0  0 ...   0   0   0   0   0   0   0   0   
1178  21584  379  0  0  0  0  0  0  0  0 ...   0   0   0   0   0   0   0   0   
59    20465  304  0  0  0  0  0  0  0  0 ...   0   0   0   0   0   0   0   0   

      28  29  
667    0   0  
1178   0   0  
59     0   0  

[3 rows x 32 columns] 

[[20465, 5.0, "dr. laroque is awesome!i broke my wrist in a bike race. a hairline fracture on the ulna, nondisplaced, it doesn't hurt, it's barely swollen  it doesn't seem like a big deal, i only went to see a doctor because it seemed a little odd that i couldn't rotate my arm much or use much force with my hand. i went to see my pcp at pamf in mountain view wh

In [99]:
with open("Topic17_RR1.txt", "w") as text_file:
    text_file.write(write1)
with open("Topic17_RR2.txt", "w") as text_file:
    text_file.write(write2)  

In [100]:
print write1

[20609, 2.0, "I had a lot of pain in my left leg, so I did some yelping, checked my medical plan, and made an appointment with Dr. Akizuki. The doctor spent as much time initially with me as seemed necessary. He didn't rush, and explained he thought I had a hip problem. I was to get an xrays and an MRI and call back.Once the MRI results were back, Dr. Akizuki and I began a game of phone tag. Finally in a voice mail message, he told me that my ankle was in pretty bad shape. We'd need an appointment to sort out what to do. It was a bit annoying to have to exchange two or three phone calls to get this information, and of course, I was a bit surprised that the diagnosis had changed from a hip problem to an ankle problem. In fact, I was sure a mix up had occurred. I made another appointment.Dr. Akizuki explained that yes it was a hip problem no explanation about the ankle message and that I was a bit young for hip replacement although he didn't rule that out. He said we could try cortisone 

In [101]:
print write2

[23355, 5.0, "I took my son to Dr. Saxena following a ruptured achilles tendon. He is an elite level athlete with college opportunities slipping with every week off field. BUT walking the rest of his life without a limp is most important! I found Dr. Saxena confident in his diagnosis and matter of fact in his prognosis. It was in his bio online with Palo Alto Medical Foundation that he specialized in Achilles work. So though he was the only Dr. available for consult Friday morning, I believe he was the right doctor. He came in before his scheduled surgeries to meet with us. We raced to get there so we would not be the cause of a late doctor. He did not rush through his exam. He spoke to my son, directly! He understood that as a junior in high school he was lookng at college and asked which ones. I booked my son's surgery for the following Tuesday. As an anxious mom, I sought a second opinion from a highly regarded orthopedist. He was wonderful too. Nice. But no more conversational than

In [92]:
with open("Topic22_RR2.txt", "r") as text_file:
    lines = text_file.read()  #type string
print type(lines)
print lines


<type 'str'>
[21156, 5.0, "I had a total hip replacement with Dr Callander last year on 129, so I'm over 1 year post op. Having the total hip completely changed my life! I'm only in my 40's but have had two hip surgeries prior to the total hip prior surgeries were not done by Dr. Callander. I was diagnosed with congenital hip dysplasia in my late 20's it was never diagnosed when I was younger. It's important to note that growing up I was extremely athletic  soccer, softball, track, etc. so to go from very active to literally not being able to walk normal in my late 20's was horrific. I visited several doctors before I was correctly diagnosed. My first surgery was around age 30 and because I was so young they opted NOT to do a total hip at that time and did a Periacetabular Osteotmy. This actually made my following years very difficult. In the end I don't think that was the right surgery for me, as it completely limited my range of motion and I still walked with an extremely pronounced 

In [83]:
def most_representative_review(mytopic,Reviews_topics_df):
    """ What review best represents a given topic?
    Selects the review with the highest gamma-weights on one topic.
    Accepts mytopic (an int corresponding to topic #) and the Rev x G matrix with rid, bid and gammas"""

    #print(ids_gammas_df.head())#Check what df looks like prior to sorting

    # Sort on a given topic
    mysorted = Reviews_topics_df.sort(mytopic, axis=0, ascending=False)
    idx = mysorted.index[0]# get index of top-ranking review for mytopic

    RID = int(mysorted.loc[idx]['RID'])
    print 'Index = ',idx, ' Starting RID: ',RID,' Topic: ',mytopic,'\n'#Note that if you replace idx with 0, and print the results, it is not the same
    #print(mysorted.loc[0]['RID'])# does not work

    #from KAsql import query_SQL
    sql = 'SELECT id, stars, comment FROM review WHERE id='+str(RID)+';'
    rows = query_SQL(sql)

    sqlid = rows[0][0]
    stars = rows[0][1]
    comment = rows[0][2]
    
    #Print statements
    print sql,'\n'
    print '\nTop review most related to topic: ',mytopic,'\n'
    print(mysorted.head(3))
    print str(RID), sqlid, stars, comment

In [84]:
most_representative_review(17,ids_gammas_df)

Index =  4901  Starting RID:  17987  Topic:  17 

USE ortho;


IndexError: tuple index out of range

In [10]:
pwd

u'/Users/kaschbacher/insightproject/yelp/git-yelp'